In [ ]:
import json
import networkx as nx
from ipysigma import Sigma

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

In [ ]:
from csv import DictReader

flows = []
with open('../../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['customs_region'] == 'Marseille' \
            and row['year'] == '1789'\
            :
            flows.append(dict(row))
print(len(flows))

In [ ]:
# créer un graphe
Graph = nx.Graph()

# créer des dict pour les deux types de noeuds et les liens
products_uniq = {}
partners_uniq = {}
edges_uniq = {}
# remplir les dicts
for flow in flows:
    partner_id = flow["partner_simplification"]
    product_id = flow["product_RE_aggregate"]
    if product_id in products_uniq:
        products_uniq[product_id] = {**products_uniq[product_id], "size": products_uniq[product_id]["size"] + 1}
    else:
       products_uniq[product_id] = {
           "type": "product", 
           "name": product_id, 
           "color": "rgb(0, 255, 0)",
           "size": 1
       }
    if partner_id in partners_uniq:
        partners_uniq[partner_id] = {**partners_uniq[partner_id], "size": partners_uniq[partner_id]["size"] + 1}
    else:
       partners_uniq[partner_id] = {
           "type": "partner", 
           "name": partner_id, 
           "color": "rgb(255, 0, 0)",
           "size": 1
       }
    edge_footprint = partner_id + "-" + product_id
    if edge_footprint in edges_uniq:
        edges_uniq[edge_footprint]["weight"] += 1
    else: 
        edges_uniq[edge_footprint] = {
            "source": product_id,
            "target": partner_id,
            "weight": 1
        }
# concaténer les deux dicts de noeuds en un seul
all_nodes = partners_uniq
all_nodes.update(products_uniq)
# applatir et formatter les noeuds
nodes = []
for key, node in all_nodes.items():
    nodes.append((key, node))
edges = []

for key, edge in edges_uniq.items():
    edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
    
domain_min_nodes_size = min([node[1]['size'] for node in nodes])
domain_max_nodes_size = max([node[1]['size'] for node in nodes])
range_in_nodes_size = [1, 10]
nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

for node in nodes:
    node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
    node[1]["label"] = node[1]["name"]

Graph.add_nodes_from(nodes)
Graph.add_edges_from(edges)

# nx.write_gexf(Graph, 'text_graph.gexf')

In [ ]:
# Sigma(Graph, node_size='size', edge_size ='weight', node_color='color')
Sigma(Graph)